# 分类问题

## 手写数字识别

**图片的表示方法**

一张图片包含了ℎ行(Height/Row)，𝑤列
(Width/Column)，每个位置保存了像素(Pixel)值，像素值一般使用 0~255 的整形数值来表达
颜色强度信息，例如 0 表示强度最低，255 表示强度最高。如果是彩色图片，则每个像素
点包含了 R、G、B 三个通道的强度信息，分别代表红色通道、绿色通道、蓝色通道的颜
色强度，所以与灰度图片不同，它的每个像素点使用一个 1 维、长度为 3 的向量(Vector)来
表示，向量的 3 个元素依次代表了当前像素点上面的 R、G、B 颜色强值，因此彩色图片
需要保存为形状是[ℎ, 𝑤, 3]的张量(Tensor，可以通俗地理解为 3 维数组)。如果是灰度图
片，则使用一个数值来表示灰度强度，例如 0 表示纯黑，255 表示纯白，因此它只需要一
个形状为[ℎ, 𝑤]的二维矩阵(Matrix)来表示一张图片信息(也可以保存为[ℎ, 𝑤, 1]形状的张
量)。

我们用形状为[ℎ, 𝑤]的矩阵来表示一张图片，
对于多张图片来说，我们在前面添加一个数量维度(Dimension)，使用形状为[𝑏, ℎ, 𝑤]的张量
来表示，其中𝑏代表了批量(Batch Size)；多张彩色图片可以使用形状为[𝑏, ℎ, 𝑤, 𝑐]的张量来
表示，其中𝑐表示通道数量(Channel)，彩色图片𝑐 = 3。通过 TensorFlow 的 Dataset 对象可
以方便完成模型的批量训练，只需要调用 batch()函数即可构建带 batch 功能的数据集对
象。

In [1]:
import os
import tensorflow as tf
from tensorflow.keras import layers, optimizers, datasets


In [2]:
# 训练集 测试集
(x, y), (x_val, y_val) = datasets.mnist.load_data()  # 加载 MNIST 数据集

In [3]:
x.shape  # 60000个样本 每个28行28列

(60000, 28, 28)

In [4]:
# 转为浮点张量 归一化 再缩放到-1 ~ 1
x = 2 * tf.convert_to_tensor(x, dtype=tf.float32) / 255. - 1

In [5]:
x[1]

<tf.Tensor: shape=(28, 28), dtype=float32, numpy=
array([[-1.        , -1.        , -1.        , -1.        , -1.        ,
        -1.        , -1.        , -1.        , -1.        , -1.        ,
        -1.        , -1.        , -1.        , -1.        , -1.        ,
        -1.        , -1.        , -1.        , -1.        , -1.        ,
        -1.        , -1.        , -1.        , -1.        , -1.        ,
        -1.        , -1.        , -1.        ],
       [-1.        , -1.        , -1.        , -1.        , -1.        ,
        -1.        , -1.        , -1.        , -1.        , -1.        ,
        -1.        , -1.        , -1.        , -1.        , -1.        ,
        -1.        , -1.        , -1.        , -1.        , -1.        ,
        -1.        , -1.        , -1.        , -1.        , -1.        ,
        -1.        , -1.        , -1.        ],
       [-1.        , -1.        , -1.        , -1.        , -1.        ,
        -1.        , -1.        , -1.        , -1. 

In [6]:
# 转换为整形张量
y = tf.convert_to_tensor(y, dtype=tf.int32)
y[:10]

<tf.Tensor: shape=(10,), dtype=int32, numpy=array([5, 0, 4, 1, 9, 2, 1, 3, 1, 4], dtype=int32)>

In [7]:
# one-hot 独热码
y = tf.one_hot(y, depth=10)
y[:10]

<tf.Tensor: shape=(10, 10), dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]], dtype=float32)>

## 模型构建
对于多输出节点、批量训练方式，我们将模型写成批量形式：
$$Y = X@W + b$$
其中$𝑿 \in 𝑅^{b × 𝑑_{in}}，𝒃 ∈ 𝑅^{d_{out}}，𝒀 ∈ 𝑅^{b \times d_{out}}，𝑾 \in R^{d_{in} \times d_{out}}，d_{in}表示输入节点数，d_{out}表示输出节点数$
输出节点数𝑿形状为$[b, d_{in}]$，表示𝑏个样本的输入数据，每个样本的特征长度为$d_{in}$

输入特征长度$d_{in} = 3，输出特征长度d_{out} = 2$的模型

![模型](../assets/3输入2输出模型.png)

### one-hot编码

In [8]:
y = tf.constant([0, 1, 2, 3])
y = tf.one_hot(y, depth=10)  # 指定类别总数为 10
y

<tf.Tensor: shape=(4, 10), dtype=float32, numpy=
array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.]], dtype=float32)>

输入是一张展平后的图片向量$x \in R^{784}$, 输出是长度为10的向量$o \in R^{10}$

预测模型采用多输入、多输出的线性模型 $o = 𝑾^𝐓𝒙 + 𝒃，$
其中模型的输出记为输入的预测值 ，我们希望 越接近真实标签𝒚越好。一般把输入经过
一次(线性)变换叫作一层网络。

## 误差计算
对于分类问题来说，我们的目标是最大化某个性能指标，比如准确度𝑎𝑐𝑐，但是把准确度当作损失函数去优化时发现$\frac {\partial{acc}} {\partial \theta}$不可导, 无法利用梯度下降算法优化网络参数.一般的做法是，设立一个平滑可导的代理目标函数，比如优化模型的输出 与 One-hot 编码后的真实标签𝒚之间的**距离**(Distance)，通过优化代理目标函数得到的模型，一般在测试性能上也能有良好的表现。因此，相对回归问题而言，分类问题的优化目标函数和评价目标函数是不一致的。
$$W^*, b^* = argmin_{w, b} L(o, y)$$

对n个样本的**均方差损失函数**可以表达为
$$L(o, y) = \frac 1 n \sum_{i=1}^n\sum_{j=1}^{10}(o_j^{(i)} - y_j^{(i)})^2$$

## 存在的问题
1. 线性模型过于简单, 逼近复杂的人脑图片识别模型，很显然不能胜任
1. 表达能力, 表达能力体现为逼近复杂分布的能力。上面的解决方案只使用了少量神经元组成的一层网络模型，相对于人脑中千亿级别的神经元互联结构，它的表达能力明显偏弱


## 解决与优化
1. 给线性模型嵌套一个非线性函数, 将其转为非线性模型, 我们把这个非线性函数称为激活函数(Activation Function)，用𝜎表示:
$$o = \sigma(Wx + b)$$